In [1]:
from concurrent.futures import ThreadPoolExecutor
from typing import List, Dict, Any, Type

from environmental_risk_metrics import (
    Sentinel2,
    EsaLandCover,
    EsriLandCover,
    OpenLandMapLandCover,
    SoilOrganicCarbon,
    SoilTypes,
    EndangeredSpecies,
    RamsarProtectedAreas,
    GlobalWitness,
)

polygon = {
    "type": "Feature",
    "properties": {},
    "geometry": {
        "coordinates": [
            [
                [10.235198982658801, 51.42076009745068],
                [10.236477278753114, 51.41697045550828],
                [10.244461712820623, 51.41823370440062],
                [10.242888425319222, 51.4220355049745],
                [10.235198982658801, 51.42076009745068],
            ]
        ],
        "type": "Polygon",
    },
}

start_date = "2020-05-01"
end_date = "2024-08-31"

polygon_crs = "EPSG:4326"

def calculate_metrics(
    metric_classes: List[Type],
    polygon: dict,
    start_date: str,
    end_date: str,
    polygon_crs: str = "EPSG:4326",
    timeout: float = None
) -> Dict[str, Any]:
    """
    Calculate multiple environmental metrics in parallel.
    
    Args:
        metric_classes: List of metric classes to calculate
        polygon: GeoJSON polygon
        start_date: Start date string (YYYY-MM-DD)
        end_date: End date string (YYYY-MM-DD)
        polygon_crs: CRS of the input polygon
        timeout: Maximum time in seconds to wait for each metric
    
    Returns:
        Dictionary with metric names as keys and their results as values
    """
    def calculate_single_metric(metric_class: Type) -> tuple[str, Any]:
        try:
            metric = metric_class()
            result = metric.get_data(
                polygon=polygon,
                start_date=start_date,
                end_date=end_date,
                polygon_crs=polygon_crs
            )
            return metric_class.__name__, result
        except Exception as e:
            return metric_class.__name__, {"error": str(e)}

    # Run all metrics in parallel
    with ThreadPoolExecutor() as executor:
        futures = [
            executor.submit(calculate_single_metric, metric_class)
            for metric_class in metric_classes
        ]
        
        return dict(
            future.result(timeout=timeout)
            for future in futures
        )

# Example usage:
metrics = [
    Sentinel2,
    EsaLandCover,
    EsriLandCover,
    OpenLandMapLandCover,
    SoilOrganicCarbon,
    SoilTypes,
    EndangeredSpecies,
    RamsarProtectedAreas,
    GlobalWitness,
]

results = calculate_metrics(
    metric_classes=metrics,
    polygon=polygon,
    start_date=start_date,
    end_date=end_date,
    timeout=300  # 5 minutes timeout
)

# Display results
for metric_name, result in results.items():
    print(f"\n{metric_name}:")
    display(result)

/Users/thimmzwiener/risk-reports/.venv/lib/python3.12/site-packages/xarray/core/duck_array_ops.py:226: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)
  0%|          | 0/48 [00:00<?, ?it/s]
/Users/thimmzwiener/risk-reports/.venv/lib/python3.12/site-packages/rasterio/warp.py:387: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dest = _reproject(
/Users/thimmzwiener/risk-reports/.venv/lib/python3.12/site-packages/rasterio/warp.py:387: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
 52%|█████▏    | 25/48 [00:00<00:00, 36.99it/s]

  0%|          | 0/330 [00:00<?, ?it/s]

100%|██████████| 48/48 [00:01<00:00, 28.95it/s]



Sentinel2:


,mean_ndvi
2020-05-01,0.885954
2020-05-02,0.885954
2020-05-03,0.885954
2020-05-04,0.885954
2020-05-05,0.885954
...,...
2024-08-27,0.114095
2024-08-28,0.111633
2024-08-29,0.111633
2024-08-30,0.111633


...

,kingdom,class,species,iucnRedListCategory
0,Plantae,Bryopsida,Brachytheciastrum velutinum,Not Evaluated
1,Plantae,Polypodiopsida,Dryopteris filix-mas,Not Evaluated
2,Plantae,Bryopsida,Mnium hornum,Not Evaluated
3,Plantae,Magnoliopsida,Stellaria media,Not Evaluated
4,Plantae,Magnoliopsida,Rumex acetosa,Not Evaluated
...,...,...,...,...
277,Animalia,Insecta,Harmonia axyridis,Not Evaluated
278,Animalia,Mammalia,Meles meles,Least Concern
284,Animalia,Insecta,Phlogophora meticulosa,Not Evaluated
285,Plantae,Magnoliopsida,Achillea millefolium,Least Concern



RamsarProtectedAreas:


[{'name': 'Helmestausee Berga-Kelbra',
  'distance_km': 84.66,
  'description': "Helmestausee Berga-Kelbra. 31/07/78; Thüringen, Sachsen-Anhalt; 1,453 ha; 51°26'N 011°00'E. Special Protection Area EC Directive; Landscape Protection Areas, Nature Protection Area, Protected Area for Waterfowl. Located in the Helme River floodplain, the site consists of a flood protection reservoir, associated reedbeds and frequently flooded wet meadows. The area includes saline springs and salt meadows surrounded by pasture and agricultural fields. The site is internationally important for wintering and staging several species of waterbirds. A few species of birds breed at the site and use the area for roosting. Human activities include intensive fish-farming, recreation and nature education. A bird observatory is located at the site. Ramsar site no. 176. Most recent RIS information: 2001.",
  'ramsar_id': 176},
 {'name': 'Steinhuder Meer',
  'distance_km': 214.35,
  'description': "Steinhuder Meer. 26/0


GlobalWitness:


{'total_incidents': 0, 'years': [], 'countries': ['Germany']}